In [ ]:
!pip install timm tensorflow scikit-learn seaborn tf2onnx onnx onnxruntime


In [ ]:
from google.colab import drive
import os
import numpy as np
import tensorflow as tf

# Mount the drive to access your dataset
drive.mount('/content/drive')

# Dataset path (update with your actual path)
DATA_ROOT = "/content/drive/MyDrive/datasetfinal/grayscale"   # Update path here
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
TEST_DIR  = os.path.join(DATA_ROOT, "test")

# Image size for model input (160x160 works well for RT1170)
IMG_SIZE = (160, 160)
BATCH = 16
SEED = 42

# Get class names from directory structure
class_names = sorted([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])
NUM_CLASSES = len(class_names)

# Load train and test datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    class_names=class_names,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=True,
    seed=SEED
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    class_names=class_names,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=False
)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache("/tmp/train_cache").prefetch(AUTOTUNE)
test_ds = test_ds.cache("/tmp/test_cache").prefetch(AUTOTUNE)

print("Dataset loaded:", len(train_ds), "training batches and", len(test_ds), "test batches.")


In [ ]:
!pip install timm


In [ ]:
import timm
from tensorflow.keras import layers, Model
import tensorflow as tf # Ensure tf is imported for MobileNetV2

# Build the model
def build_model():
    # Use tf.keras.applications.MobileNetV2 directly
    base = tf.keras.applications.MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
    base.trainable = False  # Freeze the base layers

    # Model input layer
    inp = layers.Input(shape=IMG_SIZE + (3,), name="input")
    x = layers.RandomFlip("horizontal")(inp)
    x = layers.RandomRotation(0.05)(x)  # Small rotation for augmentation
    x = layers.RandomZoom(0.1)(x)  # Small zoom for augmentation
    x = layers.Lambda(lambda x: (x - 127.5) / 127.5)(x)  # Normalize inputs to [-1, 1]
    x = base(x)
    x = layers.GlobalAveragePooling2D()(x)  # Global Pooling layer
    x = layers.Dropout(0.2)(x)  # Dropout to reduce overfitting
    out = layers.Dense(NUM_CLASSES, activation="softmax", name="output")(x)

    # Define model
    model = Model(inp, out, name="MobileNetV2_Classifier")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Create the model
shufflenet_model = build_model()
shufflenet_model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "MobileNetV2_Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_2 (RandomFlip)      │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_2               │ (None, 160, 160, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_2 (RandomZoom)      │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_2 (Lambda)               │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 8)              │        10,248 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,268,232 (8.65 MB)

 Trainable params: 10,248 (40.03 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
checkpoint_path = "/content/efficientnet_lite_best.keras"
cbs = [
    ModelCheckpoint(checkpoint_path, monitor="val_accuracy", save_best_only=True, verbose=1),
    EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1),
]

# Train EfficientNet Lite model
history = efficientnet_lite_model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10,
    callbacks=cbs,
    verbose=1
)

print("Best EfficientNet Lite model saved:", checkpoint_path)


Epoch 1/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 620ms/step - accuracy: 0.1626 - loss: 2.0506
Epoch 1: val_accuracy improved from -inf to 0.55359, saving model to /content/efficientnet_lite_best.keras
149/149 ━━━━━━━━━━━━━━━━━━━━ 155s 879ms/step - accuracy: 0.1628 - loss: 2.0502 - val_accuracy: 0.5536 - val_loss: 1.8021 - learning_rate: 0.0010
Epoch 2/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 615ms/step - accuracy: 0.2323 - loss: 1.9061
Epoch 2: val_accuracy did not improve from 0.55359
149/149 ━━━━━━━━━━━━━━━━━━━━ 134s 828ms/step - accuracy: 0.2323 - loss: 1.9059 - val_accuracy: 0.5467 - val_loss: 1.7286 - learning_rate: 0.0010
Epoch 3/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 591ms/step - accuracy: 0.2457 - loss: 1.8533
Epoch 3: val_accuracy improved from 0.55359 to 0.56244, saving model to /content/efficientnet_lite_best.keras
149/149 ━━━━━━━━━━━━━━━━━━━━ 122s 818ms/step - accuracy: 0.2458 - loss: 1.8532 - val_accuracy: 0.5624 - val_loss: 1.7047 - learning_rate: 0.0010
Epoch 4/10
149/149 ━━━━━━━━━━━━━━

In [ ]:
import tf2onnx

ONNX_PATH = "/content/efficientnet_lite_rt1170_160.onnx"
spec = (tf.TensorSpec((1, 160, 160, 3), tf.float32, name="input"),)

model_proto, _ = tf2onnx.convert.from_keras(
    best_efficientnet_lite_model,
    input_signature=spec,
    opset=13,
    output_path=ONNX_PATH
)

print("✅ ONNX saved:", ONNX_PATH)
print("Size MB:", os.path.getsize(ONNX_PATH) / 1024 / 1024)
